In [1]:
import sys
 
sys.path.insert(0, "../")

from EnsembleFramework import Framework

In [2]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

dataset_name = 'Cora'
split = "public"
dataset = Planetoid(root='/tmp/Cora', name=dataset_name, split=split)
dataset.transform = T.NormalizeFeatures()

features =  dataset[0].x
y =  dataset[0].y

test =  dataset[0].test_mask
train = dataset[0].train_mask
val =  dataset[0].val_mask

edge_index = dataset[0].edge_index 
edge_index = add_self_loops(edge_index)[0]

In [3]:
hops_to_best_configs = {0: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 1: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 2: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00011111111111111112},
 3: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00022222222222222223},
 4: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 5: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 6: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0003333333333333334},
 7: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 8: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 9: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00022222222222222223},
 10: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 15: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 20: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 30: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00044444444444444447},
 40: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0003333333333333334},
 50: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0}}

In [5]:
from torch.nn.functional import normalize
def user_function(kwargs):
    return  normalize(kwargs["original_features"] + kwargs["summed_neighbors"], p=2.0, dim = 1)
    
hops_list = [0,1,2,3,4,5,6,7,8,9,10,15,20,30,40,50]
framework = Framework([user_function for i in hops_list], 
                     hops_list=hops_list, ## to obtain best for local neighborhood
                     clfs=[],
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=[hops_to_best_configs[hop] for hop in hops_list])
new_features_list = framework.get_features(features, edge_index, None)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm
from hyperopt import fmin, tpe, hp,STATUS_OK

n_estimators = [50, 100,150, 200, 400, 500, 800, 1_000, 1_200]
max_depth = [None, 2,3,4,5,8,10]
# max_delta_step = [None, 1, 2, 3, 4]

lr_choices = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    # 'max_delta_step': max_delta_step,
}

space = {
    **{key: hp.choice(key, value) for key, value in lr_choices.items()},
    'min_child_weight': hp.uniform('min_child_weight', 0, 5),
    'eta': hp.loguniform('eta', -3, -.3),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
}

clf = XGBClassifier
hops_to_params=dict({})
hops_to_score=dict({})

for i, hops in tqdm(enumerate(hops_list)):
    new_features = new_features_list[i].cpu()
    
    def objective(params):
        score = 0
        try:
            model = clf(**params, random_state = 42,  tree_method = "hist", device="cuda:0")
            model.fit(new_features[train], y[train])
            y_pred = model.predict(new_features[val])
            score = accuracy_score(y[val], y_pred)
        except Exception:
            print(Exception)
            print(params)
            score = -100
        return {'loss': -score, 'status': STATUS_OK}
    
    best_params = fmin(objective, space, algo=tpe.suggest, max_evals=1_000, verbose=1)

    for key,value in lr_choices.items():
        best_params[key] = value[best_params[key]]

    best_model = clf(**best_params, random_state = 42,  tree_method = "hist", device="cuda:0")
    best_model.fit(new_features[train], y[train])
    y_pred = best_model.predict(new_features[val])
    
    hops_to_params[hops] = best_params
    hops_to_score[hops] = accuracy_score(y[val], y_pred)

0it [00:00, ?it/s]


100%|█████████████████████████████████████████████████████████| 1000/1000 [33:27<00:00,  2.01s/trial, best loss: -0.552]

100%|██████████████████████████████████████████████████████████| 1000/1000 [31:37<00:00,  1.90s/trial, best loss: -0.72]

100%|██████████████████████████████████████████████████████████| 1000/1000 [27:24<00:00,  1.64s/trial, best loss: -0.77]

100%|█████████████████████████████████████████████████████████| 1000/1000 [19:48<00:00,  1.19s/trial, best loss: -0.798]

100%|█████████████████████████████████████████████████████████| 1000/1000 [26:34<00:00,  1.59s/trial, best loss: -0.806]

100%|█████████████████████████████████████████████████████████| 1000/1000 [20:56<00:00,  1.26s/trial, best loss: -0.814]

100%|█████████████████████████████████████████████████████████| 1000/1000 [37:03<00:00,  2.22s/trial, best loss: -0.816]

100%|█████████████████████████████████████████████████████████| 1000/1000 [23:35<00:00,  1.42s/trial, best loss: -0.806]

100%|██████████████████

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["best_params"], index=hops_to_params.keys())
df["best_params"] = hops_to_params.values()
df.to_csv("normalized_origin_features_plus_summed_neighbors.csv")
df

In [ ]:
hops_to_params

In [ ]:
hops_to_score